In [2]:
import pandas as pd
import kagglehub


# Last ned datasettene
pathGermany = kagglehub.dataset_download("l3llff/wind-power")

# Les inn CSV-filene som DataFrames
df_germany = pd.read_csv(f"{pathGermany}/data.csv")


/Users/juliehe/Documents/Miljøanalyse/Mappeprosjekt_Milj-dataanalyse/gruppe_venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importerer det jeg trenger til oppgave 3, del 1

In [17]:
#Undersøker om det finnes manglede dager

df_germany['dt'] = pd.to_datetime(df_germany['dt'], errors='coerce')

df_germany['d'] = df_germany['dt'].dt.date

alle_dager = pd.date_range(start=df_germany['d'].min(), end=df_germany['d'].max(), freq='d')
manglende_dager = alle_dager.difference(df_germany['d'])

if not manglende_dager.empty:
    print(f"Følgende dager mangler: {manglende_dager}")
else:
    print("Ingen manglende dager")


Ingen manglende dager


Ser at det er ingen hele dager som mangler

In [ ]:

#både dato og tid 
df_germany['datetime'] = df_germany['dt']

# tidsrekke med 15-minutters intervaller
alle_intervaller = pd.date_range(start=df_germany['datetime'].min(), 
                                 end=df_germany['datetime'].max(), 
                                 freq='15T')

# Finner de manglende intervaller
manglende_intervaller = alle_intervaller.difference(df_germany['datetime'])

# Sjekker om det er noen manglende intervaller
if not manglende_intervaller.empty:

    # Lager en DataFrame for de manglende intervallene
    manglende_intervaller_df = pd.DataFrame(manglende_intervaller, columns=['Manglende intervaller'])

    print("\nFølgende 15-minutters intervaller mangler:\n")
    print(manglende_intervaller_df.to_string(index=False))  
    print(f"\nAntall manglende 15-minutters intervaller: {len(manglende_intervaller_df)}")
    # Lagrer resultatet som en CSV-fil for videre bruk
    manglende_intervaller_df.to_csv("manglende_intervaller.csv", index=False)
    print("\nManglende intervaller er også lagret i 'manglende_intervaller.csv'.")
else:
    print("Ingen manglende 15-minutters intervaller") #Om det ikke er manglende verdier 



Følgende 15-minutters intervaller mangler:

Manglende intervaller
  2011-03-27 02:00:00
  2011-03-27 02:15:00
  2011-03-27 02:30:00
  2011-03-27 02:45:00
  2012-03-25 02:00:00
  2012-03-25 02:15:00
  2012-03-25 02:30:00
  2012-03-25 02:45:00
  2013-03-31 02:00:00
  2013-03-31 02:15:00
  2013-03-31 02:30:00
  2013-03-31 02:45:00
  2014-03-30 02:00:00
  2014-03-30 02:15:00
  2014-03-30 02:30:00
  2014-03-30 02:45:00
  2014-04-07 03:30:00
  2014-06-20 14:45:00
  2014-06-20 15:30:00
  2014-06-25 11:30:00
  2014-06-25 13:45:00
  2014-06-25 14:15:00
  2015-03-29 02:00:00
  2015-03-29 02:15:00
  2015-03-29 02:30:00
  2015-03-29 02:45:00
  2016-03-27 02:00:00
  2016-03-27 02:15:00
  2016-03-27 02:30:00
  2016-03-27 02:45:00
  2017-03-26 02:00:00
  2017-03-26 02:15:00
  2017-03-26 02:30:00
  2017-03-26 02:45:00
  2018-03-25 02:00:00
  2018-03-25 02:15:00
  2018-03-25 02:30:00
  2018-03-25 02:45:00
  2018-11-26 12:15:00
  2018-11-26 12:30:00
  2019-03-31 02:00:00
  2019-03-31 02:15:00
  2019-03

/var/folders/8f/nwn4bqq56715k1k5jg6vggg00000gn/T/ipykernel_58094/1649451051.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  alle_intervaller = pd.date_range(start=df_germany['datetime'].min(),


Nå ser vi at det er 52 15-minutters intervaller som mangler. 



In [42]:

# Sorter dataene etter tid for å få riktig rekkefølge
df_germany = df_germany.sort_values('dt').reset_index(drop=True)

#tidsrekke med 15-minutters intervaller
alle_intervaller = pd.date_range(start=df_germany['dt'].min(), 
                                 end=df_germany['dt'].max(), 
                                 freq='15T')

# Finner manglende intervaller
manglende_intervaller = alle_intervaller.difference(df_germany['dt'])

# Lager en DataFrame for de manglende verdiene
manglende_data = pd.DataFrame([
    {
        'dt': tidspunkt,
        'MW': (df_germany.loc[df_germany['dt'] < tidspunkt, 'MW'].iloc[-1] +  
               df_germany.loc[df_germany['dt'] > tidspunkt, 'MW'].iloc[0]) / 2
    }
    for tidspunkt in manglende_intervaller
])

# Setter verdiene sammen 
df_komplett = pd.concat([df_germany, manglende_data]).sort_values('dt').reset_index(drop=True)

# Fjerner de siste 3 kolonnene hvis de eksisterer
if df_komplett.shape[1] > 2:
    df_komplett = df_komplett.iloc[:, :-2]

# Lagre resultatet til CSV
df_komplett.to_csv("komplett_data_med_utfylte_mengder.csv", index=False)
print("Data med utfylte verdier er lagret som 'komplett_data_med_utfylte_mengder.csv'.")


/var/folders/8f/nwn4bqq56715k1k5jg6vggg00000gn/T/ipykernel_58094/3006460994.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  alle_intervaller = pd.date_range(start=df_germany['dt'].min(),


Data med utfylte verdier er lagret som 'komplett_data_med_utfylte_mengder.csv'.


In [44]:
#Importerer den nye csv filen for å kunne bruke den videre:

df_komplett = pd.read_csv("komplett_data_med_utfylte_mengder.csv")
#Konverter 'dt'-kolonnen tilbake til datetime-format
df_komplett['dt'] = pd.to_datetime(df_komplett['dt'])
